### First run the function chunk below, then use these for generating dates outputs

## Comedy Shows

for later reading... https://www.geeksread.com/how-to-extract-data-from-json-in-javascript/

* Show Name
* Show Date
* Show Start Time
* Show Location: Venue Name
* Show Location: Street Address
* Show Location: City/Town
* Show Location: Zip
* Show Location: Country
* Show Ticket Link

It turns out that if you don't break up the location output, it will generate a string with commas in it which will certainly fuck up any future csv parsing...

In [1]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
import pandas as pd
import datetime
import re
from pprint import pprint
from calendar import monthcalendar as mc

##############################
######GETTING ALL EVENTS######
##############################

 ## SCOPES AND CREDENTIAL LOCATION CODING
SCOPES = ['https://www.googleapis.com/auth/calendar']
SERVICE_ACCOUNT_FILE = '/home/james/Documents/PythonNotebooks/Calendar API/calendar.json'

## BUILDING CREDENTIAL OBJECT
print("BUILDING CREDENTIAL OBJECT")
credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES)

## CREATING API SERVICE INSTANCE
print("CREATING API SERVICE INSTANCE")
service = build('calendar', 'v3', credentials=credentials)

## CALLING ALL EVENTS FOR THE CALENDAR FOR JAMESHUESSY@GMAIL.COM
print("CALLING ALL EVENTS FOR THE CALENDAR FOR JAMESHUESSY@GMAIL.COM")
t_now = datetime.datetime.utcnow()

d = service.events().list(calendarId='jameshuessy@gmail.com', 
                            timeMin=t_now.isoformat()+'Z'
                             )
### Calling 'execute()' is how you make the api call
dd = d.execute()

## Parsing into dataframe
print("Parsing into dataframe")
try:
    events = pd.DataFrame(dd['items'])
except Exception as err:
    print("Couldn't get items into the dataframe\nError: {}".format(err))
else:
    print("Done!")
    #print(events)


BUILDING CREDENTIAL OBJECT
CREATING API SERVICE INSTANCE
CALLING ALL EVENTS FOR THE CALENDAR FOR JAMESHUESSY@GMAIL.COM
Parsing into dataframe
Done!


In [4]:


import re
import pandas as pd

termine = pd.DataFrame({"Show_Name":[],
                        "Show_Date": [],
                        "Show_Day": [],
                        "Show_Month": [],
                        "Show_Year": [],
                        "Show_Start_Time": [],
                        "Show_Venue_Name": [],
                        "Show_Street_Address": [],
                        "Show_City": [],
                        "Show_State": [],
                        "Show_Zip": [],
                        "Show_Country": [],
                        "Ticket_Link": []}
                      )

for i in range(len(events)):
    tst = events.description[i]
    if isinstance(tst, float):
        #print("It's a float!")
        continue
    elif '#comedy' in tst:
        # Get manually entered ticket link from description if it's there
        tkt_link_chk = tst.split("\n")
        tkt_link = [x for x in tkt_link_chk if 'http' in x]
        if tkt_link:
            tkt_link = tkt_link.pop()
        else:
            tkt_link = None
        
        
        ## Getting the event ID
        e_id = events.id[i]
        ## Getting more detailed event data
        e_data = service.events().get(calendarId='jameshuessy@gmail.com', eventId=e_id).execute()
        
        ## Show Name
        e_name = e_data['summary']

        
        
        ## Show Location
        try:
            e_loc = e_data['location'].split(", ")
        except:
            e_loc = "TBA"
            print(e_name)        
        if 'TBA' not in e_loc:
            if "730 Tavern" in e_loc:
                ## This is a terrible solution, but I'm lazy for the time being
                ## if more venues have commas in their names, then I will have to 
                ## actually try to build something that can handle it with regex
                loc_venue_name = "{} {}".format(e_loc[0], e_loc[1])
                loc_street_address = e_loc[2]
                loc_city = e_loc[3]                
                loc_state = e_loc[4].split(" ")[0]
                loc_zip = e_loc[4].split(" ")[1]
                loc_country = e_loc[5]
            else:
            ## Create all vars            
                ## It looks like it follows the same pattern:
                ## [0] = Establishment Name
                ## [1] = Street Address
                ## [2] = City/Town
                ## [3] = State+Zip
                ## [4] = Country
                loc_venue_name = e_loc[0]
                loc_street_address = e_loc[1]
                loc_city = e_loc[2]
                loc_state = e_loc[3].split(" ")[0]
                loc_zip = e_loc[3].split(" ")[1]                
                loc_country = e_loc[4]
        else:
            ## Leave the vars null?
            loc_venue_name = "TBA"
            loc_street_address = None
            loc_city = None
            loc_state = None
            loc_zip = None
            loc_country = None
        
        ## Show DatTime
        e_dat_date = e_data['start']['dateTime']
        e_dat_date = re.sub(pattern=":00-.*", repl="", string= e_dat_date)
        e_dat_date = datetime.datetime.strptime(e_dat_date, '%Y-%m-%dT%H:%M')
        e_dat_date_year = int(e_dat_date.year)
        e_dat_date_month = e_dat_date.strftime("%B")
        e_dat_date_day = int(e_dat_date.day)
        
        
        ## Show Date
        e_date = e_dat_date.strftime("%x")
        
        ## Show Start Time
#         e_hour = e_dat_date.hour
#         e_min = e_dat_date.minute
#         ## dealing with lack of leading 0s on minutes
#         if e_min < 10:
#             e_min = "0{}".format(e_min)
#         e_start = "{}:{}".format(e_hour,e_min)
        e_start = e_dat_date.strftime("%I:%M %p")
        
        loader = pd.DataFrame({"Show_Name":[e_name],
                               "Show_Date": [e_date],
                               "Show_Day": [int(e_dat_date_day)],
                               "Show_Month": [e_dat_date_month],
                               "Show_Year": [e_dat_date_year],
                               "Show_Start_Time": [e_start],
                               "Show_Venue_Name": [loc_venue_name],
                               "Show_Street_Address": [loc_street_address],
                               "Show_City": [loc_city],
                               "Show_State": [loc_state],
                               "Show_Zip": [loc_zip],
                               "Show_Country": [loc_country],
                               "Ticket_Link": [tkt_link]})
        termine = termine.append(loader)

termine['Show_Day'] = pd.to_numeric(termine['Show_Day'], downcast='integer')
termine['Show_Year'] = pd.to_numeric(termine['Show_Year'], downcast='integer')
termine = termine.sort_values(by=['Show_Date', 'Show_Start_Time'], axis=0)
#print(termine)


termine.to_csv("dates.csv", index=False)

print("Done")


Done


In [22]:
termine['Show_Day'] = pd.to_numeric(termine['Show_Day'], downcast='integer')
termine
#print()


,Show_Name,Show_Date,Show_Day,Show_Month,Show_Year,Show_Start_Time,Show_Venue_Name,Show_Street_Address,Show_City,Show_State,Show_Zip,Show_Country
0,Pour House,08/14/19,14,August,2019,08:00 PM,The Pour House,907 Boylston St #21,Boston,MA,02115,USA
0,Burren,08/14/19,14,August,2019,10:00 PM,The Burren,247 Elm St,Somerville,MA,02144,USA
0,Comedy Fire,08/16/19,16,August,2019,08:00 PM,Casa Caña,1234 Soldiers Field Rd,Boston,MA,02135,USA
0,McGreevy's,08/20/19,20,August,2019,08:00 PM,McGreevy's Boston,911 Boylston St,Boston,MA,02115,USA
0,Comedy Studio,08/23/19,23,August,2019,08:00 PM,The Comedy Studio,1 Bow Market Way #23,Somerville,MA,02143,USA
0,Socks and Sandals,08/25/19,25,August,2019,08:00 PM,Half Lounge,136 1/2 Church St,Burlington,VT,05401,USA
0,Comedy and Crepes,08/26/19,26,August,2019,07:00 PM,The Skinny Pancake,60 Lake St,Burlington,VT,05401,USA
0,MA's Funniest Comedian Competition,08/28/19,28,August,2019,08:00 PM,Laugh Boston,425 Summer St,Boston,MA,02210,USA
0,Sixth Gear Comedy,08/31/19,31,August,2019,08:00 PM,Sixth Gear - Cask & Kitchen,81 L St,Boston,MA,02127,USA
0,Slumbrew,09/01/19,1,September,2019,08:00 PM,Somerville Brewing Company - Slumbrew,15 Ward St,Somerville,MA,02143,USA


In [12]:
for i in range(len(events)):
    tst = events.description[i]
    if isinstance(tst, float):
        #print("It's a float!")
        continue
    elif '#comedy' in tst:        
        ## Getting the event ID
        e_id = events.id[i]
        ## Getting more detailed event data
        e_data = service.events().get(calendarId='jameshuessy@gmail.com', eventId=e_id).execute()
        
        ## Show Name
        e_name = e_data['summary']
        ## Show Location
        try:
            e_loc = e_data['location'].split(", ")
        except:
            e_loc = "TBA"
            print(e_name)
        print(e_loc)
        if e_loc != "TBA":
            ## Create all vars            
                ## It looks like it follows the same pattern:
                ## [0] = Establishment Name
                ## [1] = Street Address
                ## [2] = City/Town
                ## [3] = State+Zip
                ## [4] = Country
            loc_street_address = e_loc[1]
            loc_city = e_loc[2]
            loc_state = e_loc[3].split(" ")[0]
            loc_zip = e_loc[3].split(" ")[1]
            loc_country = e_loc[4]
        else:
            ## Leave the vars null?
            loc_street_address = "TBA"
            loc_city = None
            loc_state = None
            loc_zip = None
            loc_country = None
    
    
    
    
    
    
    

['The Comedy Studio', '1 Bow Market Way #23', 'Somerville', 'MA 02143', 'USA']
['Casa Caña', '1234 Soldiers Field Rd', 'Boston', 'MA 02135', 'USA']
["McGreevy's Boston", '911 Boylston St', 'Boston', 'MA 02115', 'USA']
['The Comedy Studio', '1 Bow Market Way #23', 'Somerville', 'MA 02143', 'USA']
['Half Lounge', '136 1/2 Church St', 'Burlington', 'VT 05401', 'USA']
['The Skinny Pancake', '60 Lake St', 'Burlington', 'VT 05401', 'USA']
['Laugh Boston', '425 Summer St', 'Boston', 'MA 02210', 'USA']
['Somerville Brewing Company - Slumbrew', '15 Ward St', 'Somerville', 'MA 02143', 'USA']
['TBA']
['The Comedy Studio', '1 Bow Market Way #23', 'Somerville', 'MA 02143', 'USA']
['The Burren', '247 Elm St', 'Somerville', 'MA 02144', 'USA']
['ImprovBoston', '40 Prospect St', 'Cambridge', 'MA 02139', 'USA']
['The Sanctuary', '2932 Caniff St', 'Detroit', 'MI 48214', 'USA']
['Ant Hall', '2320 Caniff St', 'Hamtramck', 'MI 48212', 'USA']
['The Comedy Studio', '1 Bow Market Way #23', 'Somerville', 'MA 02

In [ ]:
## Hideout (Fiday, Saturday)

ds = free_dates(months_forward=4, booker_show="Hideout")

for i in ds:
    print(i)

In [ ]:
## Laugh/Nick's (Thursday, Fiday, Saturday)

ds = free_dates(months_forward=4, booker_show="Laugh")

for i in ds:
    print(i)

In [ ]:
## Comedy Studio (Wednesday-Sunday)

ds = free_dates(months_forward=5, booker_show="Comedy Studio")

for i in ds:
    print(i)

In [ ]:
## McGreevys (Monday, Tuesday)

ds = free_dates(months_forward=4, booker_show="McGreevys")

for i in ds:
    print(i)

In [ ]:
## Rob Crean (Wednesday, Friday)

ds = free_dates(days_var=["Wednesday", "Friday"],months_forward=4)

for i in ds:
    print(i)

In [ ]:
## Comedy Party (Thursday, Friday)

ds = free_dates(months_forward=4, booker_show="Comedy Party")

for i in ds:
    print(i)

In [ ]:
## Bill's Bar (Wednesday, Friday)

ds = free_dates(days_var=["Wednesday", "Friday"], months_forward=4)

for i in ds:
    print(i)

In [ ]:
ds = free_dates(days_var=["Saturday"], months_forward=5, monthly_week=[2])

for i in ds:
    print(i)

## WORKING FUNCTION SETS

In [ ]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
import pandas as pd
import datetime
import re
from pprint import pprint
from calendar import monthcalendar as mc



def get_the_date(week_of_month, day_of_week, year, month_num):
    days_dict = {"Monday": 0,
                 "Tuesday":1,
                 "Wednesday": 2,
                 "Thursday": 3,
                 "Friday": 4,
                 "Saturday": 5,
                 "Sunday": 6}
    possible_dates =[]
    
    for week_num in week_of_month:    
        # Get week arrays
        may = mc(year, month_num)
    
        # Narrow down to the week we're looking for
        dt = may[week_num-1]
        
        for dys in day_of_week:
            # Get the date of the day of the week we're looking for
            day_dt = dt[days_dict[dys]]
            
    
            # Combine the info into one date object
            try:
                result = datetime.date(year, month_num, day_dt)
            except:
                ## If that didn't work, then the date is probably not in range so skip it
                continue
            else:
                possible_dates += [result]
    
    
    # Send the sucker back
    return(possible_dates)




## If monthly_week: look for dates that fall on the 'days' values but also that appear in X week

def free_dates(days_var=[], months_forward=4, booker_show=None, monthly_week=None):

    
    known_shows = {"Laugh": ["Thursday", "Friday", "Saturday"],
                  "Nicks": ["Friday", "Saturday"],
                  "Hideout": ["Friday", "Saturday"],
                  "McGreevys": ["Monday", "Tuesday"],
                  "Capo": ["Monday"],
                  "Comedy Studio": ["Wednesday", "Thursday", "Friday", "Saturday", "Sunday"],
                  "Cott": ["Monday", "Thursday", "Friday", "Saturday"],
                  "Comedy Party": ["Thursday", "Saturday"]}
    
    day_dict = {0: "Monday",
                1: "Tuesday",
                2: "Wednesday",
                3: "Thursday",
                4: "Friday",
                5: "Saturday",
                6: "Sunday"}
    
    if booker_show:
        if booker_show not in known_shows.keys():
            print("The booker specified is not known\nBookers Known: {}".format(known_shows.keys()))
            return(None)
        else:
            days = known_shows[booker_show]
            
    else:
        days = days_var
    
    ####################################
    #########GETTTING GOOGLE DATA#######
    ####################################
    
    ## SCOPES AND CREDENTIAL LOCATION CODING
    SCOPES = ['https://www.googleapis.com/auth/calendar']
    SERVICE_ACCOUNT_FILE = '/home/james/Documents/PythonNotebooks/Calendar API/calendar.json'

    ## BUILDING CREDENTIAL OBJECT
    credentials = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)

    ## CREATING API SERVICE INSTANCE
    service = build('calendar', 'v3', credentials=credentials)

    ## CALLING ALL EVENTS FOR THE CALENDAR FOR JAMESHUESSY@GMAIL.COM
    t_now = datetime.datetime.utcnow()
    
    if t_now.month+months_forward > 12:
        end_year = t_now.year + 1
    elif t_now.month+months_forward <= 12:
        end_year = t_now.year
        
    t_end = datetime.datetime(end_year, t_now.month+months_forward, 1, *t_now.timetuple()[3:-2])

    d = service.events().list(calendarId='jameshuessy@gmail.com', 
                              timeMin=t_now.isoformat()+'Z',
                              timeMax=t_end.isoformat()+'Z')
    ### Calling 'execute()' is how you make the api call
    dd = d.execute()


    try:
        events = pd.DataFrame(dd['items'])
    except Exception as err:
        print("Couldn't get items into the dataframe\nError: {}".format(err))

    #print("Done")
    
    ####################################
    #########PULLING EVENTS DATES#######
    ####################################

    termine = []

    for i in range(len(events)):   
        if events['status'][i] == 'cancelled':            
            continue
            
        f = events['start'][i]
        gi = events['summary'][i]
        
        try:
            tme = f['dateTime']
        except:
            try:
                tme = events['start'][i]
                tme = tme['date']
                tme = datetime.datetime.strptime(tme, '%Y-%m-%d')
            
                tmedn = events['end'][i]
                tmedn = tmedn['date']
                tmedn = datetime.datetime.strptime(tmedn, '%Y-%m-%d')
            
                tme = datetime.datetime.date(tme)
                tmedn = datetime.datetime.date(tmedn)
            except:
                print("Couldn't parse an all day event: {}".format(gi))
        
            else:
                all_day_range = list(pd.date_range(start=tme, end=tmedn))
        
                ### Add date range dates to termine
                for i in range(len(all_day_range)-1):
                    dte = datetime.datetime.date(all_day_range[i])
                    termine += [dte]
        
        else:
            tme = re.sub(pattern='-04:00', repl="", string=tme)
        
            tme = re.sub(pattern='T', repl=" ", string=tme)
        
            tme = datetime.datetime.strptime(tme, '%Y-%m-%d %H:%M:%S')
        
            tmed = datetime.datetime.date(tme)
            try:
                tmet = datetime.datetime.time(tme)
            except:
                print("{} appears to be an all dayer".format(gi))
            else:        
                uhr19 = datetime.datetime.time(datetime.datetime.strptime("2019-05-21 18:00:00", '%Y-%m-%d %H:%M:%S'))
                if tmet > uhr19:
                    termine += [tmed]
    
    

    ####################################
    #########PULLING EVENTS DATES#######
    ####################################
    
    
    # Beginning Date (today)
    t_now = datetime.datetime.utcnow()

    # End Date (the first day of the 4th month from today)
    # fatetime.datetime() is expecting year, month, day, time, etc. which you either pass from t_now.timetupple or fill in yourself
    
    if t_now.month + months_forward > 12:
        t_year = t_now.year+1
        t_month = (t_now.month + months_forward)-12
    else:
        t_year = t_now.year
        t_month = t_now.month + months_forward
    
    t_end = datetime.datetime(t_year, t_month, 1, *t_now.timetuple()[3:-2])

    
    if monthly_week:
        t_range = []
        if t_end.month < t_now.month:
            for mnths in range(t_now.month, 12):
                # Start with the first month through december
                t_r = get_the_date(week_of_month=monthly_week,
                                   day_of_week=days,
                                   year=t_now.year, 
                                   month_num=mnths)
                t_range += [t_r]
                # Go to January through the end month of the following year
            for mnths in range(1, t_end.month):                
                t_r = get_the_date(week_of_month=monthly_week,
                                   day_of_week=days,
                                   year=t_end.year, 
                                   month_num=mnths)
                t_range += [t_r]
            
        else:
            for mnths in range(t_now.month, t_end.month):                
                t_r = get_the_date(week_of_month=monthly_week,
                                   day_of_week=days,
                                   year=t_year, 
                                   month_num=mnths)
                t_range += [t_r]
        
    
    else:
        # Date range generated
        t_range = list(pd.date_range(start=t_now, end=t_end))
        
    
    # Getting everything formatted as just the date
    for i in range(len(t_range)):
        nt = datetime.datetime.date(t_range[i])        
        t_range[i] = nt        
    
    kill = []
    for e in t_range:
        if e in termine:
            kill += [e]
    
    # If you try to kill off the dates you're already booked, you screw up the loop
    # This way you grab all the date values and then perform the remove without affecting
    # the array you're looping through. I imagine that when you got rid of an iteration,
    # it's calling it as (t_range[x]) and when you get rid of x the list moves down one.
    
    for death in kill:
        t_range.remove(death)
    
    fin = []
    for l in t_range:
        ddw = datetime.datetime.weekday(l)
        ddn = day_dict[ddw]
        if ddn in days:
            fin += [re.sub(pattern="[0-9]{4}-", repl="", string=l.isoformat())]
            
    #free_days = [x.isoformat() for x in fin]
    
    
    
    return([x for x in fin])

In [ ]:
ddg = ['2019-05-31',
 '2019-06-14',
 '2019-06-21',
 '2019-06-28',
 '2019-06-29',
 '2019-07-12',
 '2019-07-19',
 '2019-07-26',
 '2019-08-02',
 '2019-08-03',
 '2019-08-09',
 '2019-08-10',
 '2019-08-16',
 '2019-08-17',
 '2019-08-23',
 '2019-08-30',
 '2019-08-31']

for i in ddg:
    ch = re.sub(pattern="[0-9]{4}-", repl="", string=i)
    print(ch)


In [ ]:
## Get credentials set up
## Enalble Calendar API Use on account (try to use .execute() and see if it throws an error)
## Choose scope(s)
## Create credentials object
## Pass credentials object to build function to create service instance
## Use API commands to get/put/insert/etc. for calendars/events/etc.
### Use the email associated with the calendar as the calendarId to get the events from that calendar



from google.oauth2 import service_account
from googleapiclient.discovery import build
import pandas as pd
import datetime


## SCOPES AND CREDENTIAL LOCATION CODING
SCOPES = ['https://www.googleapis.com/auth/calendar']
SERVICE_ACCOUNT_FILE = '/home/james/Documents/PythonNotebooks/Calendar API/calendar.json'

## BUILDING CREDENTIAL OBJECT
credentials = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)

## CREATING API SERVICE INSTANCE
service = build('calendar', 'v3', credentials=credentials)

## CALLING ALL EVENTS FOR THE CALENDAR FOR JAMESHUESSY@GMAIL.COM
t_now = datetime.datetime.utcnow()
t_end = datetime.datetime(t_now.year, t_now.month+4, 1, *t_now.timetuple()[3:-2])
print("Starting at {} and ending at {}".format(t_now, t_end))

d = service.events().list(calendarId='jameshuessy@gmail.com', 
                          timeMin=t_now.isoformat()+'Z',
                         timeMax=t_end.isoformat()+'Z')
### Calling 'execute()' is how you make the api call
dd = d.execute()

#Pull API
#Get events data
#Parse events data
#Check for nextPageToken
#If so, re-execute service with 'nextPage=[nextPageToken]'
#repeat until you find nextSyncToken
##Probably best to do a 'while not dd[nextSyncToken]' loop





try:
    events = pd.DataFrame(dd['items'])
except Exception as err:
    print("Couldn't get items into the dataframe\nError: {}".format(err))

#stop = False
#while not stop:
#    try:
#        npt = dd['nextPageToken']
#    except Exception as err:
#        print("No next page token it would seem...\nError: {}".format(err))
#        stop = True
#        
#    else:
#        d = service.events().list(calendarId='jameshuessy@gmail.com', pageToken=npt)
#        ### Calling 'execute()' is how you make the api call
#        dd = d.execute()
#        
#        try:
#            ev_n = pd.DataFrame(dd['items'])
#        except Exception as err:
#            print("Could not create a next page dataframe from data provided\nError: {}".format(err))
#        else:
#            events = events.append(ev_n)

print("Done")
#print(events.keys())
#print(events)



#if dd['nextPageToken']:
#    while not dd['nextSyncToken']:
    

In [ ]:
import time
import re

termine = []

for i in range(len(events)):
    print("Testing event: {}".format(events['summary'][i]))
    #if events['summary'][i] == "Pittsburgh":
    #    print("i: {}".format(i))
    
    if events['status'][i] == 'cancelled':
        print('=======')
        continue
    f = events['start'][i]
    gi = events['summary'][i]
    #print(f)
    try:
        tme = f['dateTime']
    except:
        try:
            tme = events['start'][i]
            tme = tme['date']
            tme = datetime.datetime.strptime(tme, '%Y-%m-%d')
            
            tmedn = events['end'][i]
            tmedn = tmedn['date']
            tmedn = datetime.datetime.strptime(tmedn, '%Y-%m-%d')
            
            tme = datetime.datetime.date(tme)
            tmedn = datetime.datetime.date(tmedn)
        except:
            print("Couldn't parse an all day event: {}".format(gi))
        
        else:
            print("Found an all day event: {}\nStarting on {} and ending on {}".format(gi,tme,tmedn))
            all_day_range = list(pd.date_range(start=tme, end=tmedn))
        
            ### Add date range dates to termine
            for i in range(len(all_day_range)-1):
                dte = datetime.datetime.date(all_day_range[i])
                termine += [dte]
        
    else:
        tme = re.sub(pattern='-04:00', repl="", string=tme)
        
        tme = re.sub(pattern='T', repl=" ", string=tme)
        
        tme = datetime.datetime.strptime(tme, '%Y-%m-%d %H:%M:%S')
        
        tmed = datetime.datetime.date(tme)
        try:
            tmet = datetime.datetime.time(tme)
        except:
            print("{} appears to be an all dayer".format(gi))
        else:
        
            tmet = datetime.datetime.time(tme)
        
            print("Date: {}\nTime: {}".format(tmed, tmet))
        
            uhr19 = datetime.datetime.time(datetime.datetime.strptime("2019-05-21 18:00:00", '%Y-%m-%d %H:%M:%S'))
        
            if tmet > uhr19:
                print("This date seems to be a comedy show: {}".format(gi))
                termine += [tmed]
            else:
                print("This doesn't seem to conflict with comedy: {}".format(gi))
            #events['start'][i]['dateTime'] = tme
            #print(events['start'][i])
   
        
        
        
        
        ## Create ranges of these dates and add them to termine
        
        #tmed = datetime.datetime.date(tme)
        #try:
        #    tmet = datetime.datetime.time(tme)
        #except:
        #    print("{} appears to be an all dayer".format(gi))
        #else:
       # 
       #     tmet = datetime.datetime.time(tme)
        #
        #   print("Date: {}\nTime: {}".format(tmed, tmet))
        #
        #    uhr19 = datetime.datetime.time(datetime.datetime.strptime("2019-05-21 18:00:00", '%Y-%m-%d %H:%M:%S'))
        #
        #    if tmet > uhr19:
        #       print("This date seems to be a comedy show: {}".format(gi))
        #        termine += [tmed]
        #    else:
        #        print("This doesn't seem to conflict with comedy: {}".format(gi))
        
        
        
        
        
        
   # except:
   #     continue
        ##If 'date' in start, find end date, assume all dates in range are taken?
        ###yar
        #print(tme)
    print('================')
print(termine)
        
        
        
### GENERATE LIST OF DATES THAT YOU ARE BOOKED ON
### COMPARE TO A RANGE OF DATES FROM TODAY TO TODAY + ~90
### GET LOGIC GOING THAT ALLOWS YOU TO CHOOSE WHICH DAY OF THE WEEK YOU WANT TO SELECT
#### NEED TO ALLOW FOR MULTIPLE SELECTION TO INCLUDE WEEKENDS
#### MONDAY = 0, SUNDAY = 6 with datetime.datetime.weekday()


In [ ]:
from calendar import monthcalendar as mc
import datetime

## Testing week of month logic

## Assuming we know the month we're looking at we can generate an array of weeks
## These weeks are starting on Monday, which should be in line with the rest
## of the datetime functions we've been using

## Figure out what to do with year, maybe just pass it from dates/date ranges if possible

## Looking for the first Sunday

def get_the_date(week_of_month, day_of_week, year, month_num):
    days_dict = {"Monday": 0,
         "Tuesday":1,
        "Wedsday": 2,
        "Thursday": 3,
        "Friday": 4,
        "Saturday": 5,
        "Sunday": 6}
    possible_dates =[]
    
    for week_num in week_of_month:    
        # Get week arrays
        may = mc(year, month_num)
    
        # Narrow down to the week we're looking for
        dt = may[week_num-1]
        
        for dys in day_of_week:
            # Get the date of the day of the week we're looking for
            day_dt = dt[days_dict[dys]]
            
    
            # Combine the info into one date object
            try:
                result = datetime.date(year, month_num, day_dt)
            except:
                ## If that didn't work, then the date is probably not in range so skip it
                continue
            else:
                possible_dates += [result]
    
    
    # Send the sucker back
    return(possible_dates)

get_the_date(week_of_month=[1,2,3,4], day_of_week=["Monday", "Tuesday"], year=2019, month_num=5)


In [ ]:
# Beginning Date (today)
t_now = datetime.datetime.utcnow()

# End Date (the first day of the 4th month from today)
# fatetime.datetime() is expecting year, month, day, time, etc. which you either pass from t_now.timetupple or fill in yourself
t_end = datetime.datetime(t_now.year, t_now.month+4, 1, *t_now.timetuple()[3:-2])

# Date range generated
t_range = list(pd.date_range(start=t_now, end=t_end))

# Getting everything formatted as just the date
for i in range(len(t_range)):
    nt = datetime.datetime.date(t_range[i])
    #t_range[i] = nt.isoformat()
    t_range[i] = nt
    #print(t_range[i])



for r in t_range:
    if r in termine:        
        t_range.remove(r)
print("The final range of free dates is:\n{}".format(t_range))





#easily get just the dates from date objects
#print([x.isoformat() for x in t_range])

#datetime.datetime.weekday(t_range[2])

In [ ]:
day_dict = {0: "Monday",
        1: "Tuesday",
        2: "Wedsday",
        3: "Thursday",
        4: "Friday",
        5: "Saturday",
        6: "Sunday"}

days = ["Monday", "Thursday"]
days1 = ["Friday", "Thursday", "Saturday"]

fin = []
for l in t_range:
    ddw = datetime.datetime.weekday(l)
    ddn = day_dict[ddw]
    if ddn in days:
        fin += [l]
        print("You could do {}, a {}".format(l, ddn))
print(fin)
    

In [ ]:
from google.oauth2 import service_account
import pandas as pd
import datetime
import re
from pprint import pprint
from calendar import monthcalendar as mc
import re


def get_the_date(week_of_month, day_of_week, year, month_num):
    days_dict = {"Monday": 0,
                 "Tuesday":1,
                 "Wedsday": 2,
                 "Thursday": 3,
                 "Friday": 4,
                 "Saturday": 5,
                 "Sunday": 6}
    possible_dates =[]
    
    for week_num in week_of_month:    
        # Get week arrays
        may = mc(year, month_num)
    
        # Narrow down to the week we're looking for
        dt = may[week_num-1]
        
        for dys in day_of_week:
            # Get the date of the day of the week we're looking for
            day_dt = dt[days_dict[dys]]
            
    
            # Combine the info into one date object
            try:
                result = datetime.date(year, month_num, day_dt)
            except:
                ## If that didn't work, then the date is probably not in range so skip it
                continue
            else:
                possible_dates += [result]
    
    
    # Send the sucker back
    return(possible_dates)




## If monthly_week: look for dates that fall on the 'days' values but also that appear in X week

def free_dates(days=[], months_forward=4, booker_show=None, monthly_week=None):

    
    known_shows = {"Laugh": ["Thursday", "Friday", "Saturday"],
                  "Nicks": ["Friday", "Saturday"],
                  "Hideout": ["Friday", "Saturday"],
                  "McGreevys": ["Monday", "Tuesday"],
                  "Capo": ["Monday"],
                  "Comedy Studio": ["Wednesday", "Thursday", "Friday", "Saturday", "Sunday"],
                  "Cott": ["Monday", "Thursday", "Friday", "Saturday"],
                  "Comedy Party": ["Thursday", "Saturday"]}
    
    day_dict = {0: "Monday",
                1: "Tuesday",
                2: "Wedsday",
                3: "Thursday",
                4: "Friday",
                5: "Saturday",
                6: "Sunday"}
    
    if booker_show:
        if booker_show not in known_shows.keys():
            print("The booker specified is not known\nBookers Known: {}".format(known_shows.keys()))
            return(None)
        else:
            days = known_shows[booker_show]
    else:
        days = days
    
    ####################################
    #########GETTTING GOOGLE DATA#######
    ####################################
    
    ## SCOPES AND CREDENTIAL LOCATION CODING
    SCOPES = ['https://www.googleapis.com/auth/calendar']
    SERVICE_ACCOUNT_FILE = '/home/james/Documents/PythonNotebooks/Calendar API/calendar.json'

    ## BUILDING CREDENTIAL OBJECT
    credentials = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)

    ## CREATING API SERVICE INSTANCE
    service = build('calendar', 'v3', credentials=credentials)

    ## CALLING ALL EVENTS FOR THE CALENDAR FOR JAMESHUESSY@GMAIL.COM
    t_now = datetime.datetime.utcnow()
    
    if t_now.month+months_forward > 12:
        end_year = t_now.year + 1
    else:
        end_year = t_now.year
        
    t_end = datetime.datetime(end_year, t_now.month+months_forward, 1, *t_now.timetuple()[3:-2])

    d = service.events().list(calendarId='jameshuessy@gmail.com', 
                              timeMin=t_now.isoformat()+'Z',
                              timeMax=t_end.isoformat()+'Z')
    ### Calling 'execute()' is how you make the api call
    dd = d.execute()


    try:
        events = pd.DataFrame(dd['items'])
    except Exception as err:
        print("Couldn't get items into the dataframe\nError: {}".format(err))

    #print("Done")
    
    ####################################
    #########PULLING EVENTS DATES#######
    ####################################

    termine = []

    for i in range(len(events)):   
        if events['status'][i] == 'cancelled':            
            continue
            
        f = events['start'][i]
        gi = events['summary'][i]
        
        try:
            tme = f['dateTime']
        except:
            try:
                tme = events['start'][i]
                tme = tme['date']
                tme = datetime.datetime.strptime(tme, '%Y-%m-%d')
            
                tmedn = events['end'][i]
                tmedn = tmedn['date']
                tmedn = datetime.datetime.strptime(tmedn, '%Y-%m-%d')
            
                tme = datetime.datetime.date(tme)
                tmedn = datetime.datetime.date(tmedn)
            except:
                print("Couldn't parse an all day event: {}".format(gi))
        
            else:
                all_day_range = list(pd.date_range(start=tme, end=tmedn))
        
                ### Add date range dates to termine
                for i in range(len(all_day_range)-1):
                    dte = datetime.datetime.date(all_day_range[i])
                    termine += [dte]
        
        else:
            tme = re.sub(pattern='-04:00', repl="", string=tme)
        
            tme = re.sub(pattern='T', repl=" ", string=tme)
        
            tme = datetime.datetime.strptime(tme, '%Y-%m-%d %H:%M:%S')
        
            tmed = datetime.datetime.date(tme)
            try:
                tmet = datetime.datetime.time(tme)
            except:
                print("{} appears to be an all dayer".format(gi))
            else:        
                uhr19 = datetime.datetime.time(datetime.datetime.strptime("2019-05-21 18:00:00", '%Y-%m-%d %H:%M:%S'))
                if tmet > uhr19:
                    termine += [tmed]
    
    

    ####################################
    #########PULLING EVENTS DATES#######
    ####################################
    
    
    # Beginning Date (today)
    t_now = datetime.datetime.utcnow()

    # End Date (the first day of the 4th month from today)
    # fatetime.datetime() is expecting year, month, day, time, etc. which you either pass from t_now.timetupple or fill in yourself
    
    if t_now.month + months_forward > 12:
        t_year = t_now.year+1
        t_month = (t_now.month + months_forward)-12
    else:
        t_year = t_now.year
        t_month = t_now.month + months_forward
    
    t_end = datetime.datetime(t_year, t_month, 1, *t_now.timetuple()[3:-2])

    
    if monthly_week:
        t_range = []
        if t_end.month < t_now.month:
            for mnths in range(t_now.month, 12):                
                t_r = get_the_date(week_of_month=monthly_week,
                                   day_of_week=days,
                                   year=t_now.year, 
                                   month_num=mnths)
                t_range += [t_r]
            for mnths in range(1, t_end.month):                
                t_r = get_the_date(week_of_month=monthly_week,
                                   day_of_week=days,
                                   year=t_end.year, 
                                   month_num=mnths)
                t_range += [t_r]
            
        else:
            for mnths in range(t_now.month, t_end.month):                
                t_r = get_the_date(week_of_month=monthly_week,
                                   day_of_week=days,
                                   year=t_year, 
                                   month_num=mnths)
                t_range += [t_r]
        
    
    else:
        # Date range generated
        t_range = list(pd.date_range(start=t_now, end=t_end))
        
    
    # Getting everything formatted as just the date
    for i in range(len(t_range)):
        nt = datetime.datetime.date(t_range[i])        
        t_range[i] = nt        
    
    kill = []
    for e in t_range:
        if e in termine:
            kill += [e]
    
    # If you try to kill off the dates you're already booked, you screw up the loop
    # This way you grab all the date values and then perform the remove without affecting
    # the array you're looping through. I imagine that when you got rid of an iteration,
    # it's calling it as (t_range[x]) and when you get rid of x the list moves down one.
    
    for death in kill:
        t_range.remove(death)
    
    fin = []
    for l in t_range:
        ddw = datetime.datetime.weekday(l)
        ddn = day_dict[ddw]
        if ddn in days:
            fin += [l]
            
    
    
    return([x.isoformat() for x in t_range])
    
    
#free_dates(days=[], months_forward=4, booker_show="Laugh", monthly_week=None)
    
    
    

In [ ]:
kill = []
for e,w in enumerate(t_range):
    #print(e)
    if t_range[e] in termine:
        print("Yup\n{}".format(t_range[e]))        
    else:
        print("Nope\n{}".format(t_range[e]))        
#print(t_range)
#t_range.remove([x for x in kill])
#print(t_range)
print(kill)


In [ ]:
import json
import pandas as pd


t_range

termine


#events

# Get a page of events
# pull out event ID and check color
# flag ones you want to keep

#tk1 = dd['nextPageToken']

#dfg = service.events().list(calendarId='jameshuessy@gmail.com', pageToken=tk1).execute()

#ev = pd.DataFrame(dd['items'])
#ev.reindex

#print(dd['items'])
#print(dfg['items'])

#dd['items']


In [ ]:
#print(dfg.keys())
#for i in dfg.keys():
#    print(i)
#    print(dfg[i])
#    print('==========')

In [ ]:
#f[3].keys()
#len(f)
from pprint import pprint

r = f[35]['id']

g = service.events().get(calendarId='jameshuessy@gmail.com', eventId=r).execute()
#pprint(g['extendedProperties'])
#for i in g.keys():
#    print(i)
#    print(g[i])
#    print('=============')

for i in range(len(f)):
    f[i]['colorId']




In [ ]:
dfg = service.colors().get().execute()
pprint(dfg)

#10=standard
#3=maybe